In [1]:
from typing import Union

from fuzzywuzzy import fuzz, process
import numpy as np
import pandas as pd
import tqdm

from utils.regions import Regions

In [2]:
pd.set_option("display.max_columns", None)
%precision 3

'%.3f'

In [3]:
cities_base = pd.read_csv("assets/cities.csv")
cities_additional = pd.read_csv("assets/cities_additional.csv")
regions = Regions("assets/regions.csv")
rsmp = pd.read_csv("rsmp/reestr_group_A/data.csv", dtype=str)
settlements = pd.read_csv("assets/settlements.csv")
socr = pd.read_csv("assets/socrbase.csv")

In [4]:
cities = pd.concat((cities_base, cities_additional))
cities.reset_index(drop=True)
cities["id"] = range(0, cities.shape[0])

rsmp["id"] = range(0, rsmp.shape[0])

In [5]:
socr_full_to_full = socr[["name_full", "name_full"]]
socr_full_to_full.columns = ("name", "name_full")
socr_full_to_full.head(3)

,name,name_full
0,Автономный округ,Автономный округ
1,Автономная область,Автономная область
2,Город,Город


In [6]:
socr_without_dot = socr.loc[~socr["name"].str.endswith("."), ["name", "name_full"]]
socr_without_dot["name"] = socr_without_dot["name"] + "."
socr_without_dot.head(3)

,name,name_full
0,АО.,Автономный округ
1,Аобл.,Автономная область
2,г.,Город


In [7]:
abbr_to_full = pd.concat(
    (socr[["name", "name_full"]], socr_full_to_full, socr_without_dot)
)
abbr_to_full["name"] = abbr_to_full["name"].str.upper()
abbr_to_full.drop_duplicates("name", inplace=True)
abbr_to_full.head(10)

,name,name_full
0,АО,Автономный округ
1,АОБЛ,Автономная область
2,Г,Город
3,КРАЙ,Край
4,ОБЛ,Область
5,РЕСП,Республика
6,ОКРУГ,Округ
7,ЧУВАШИЯ,Чувашия
8,А.ОБЛ.,Автономная область
9,А.ОКР.,Автономный округ


In [8]:
address_cols = [
    "region_name",
    "region_type",
    "district_name",
    "district_type",
    "city_name",
    "city_type",
    "settlement_name",
    "settlement_type",
]
addresses = rsmp.loc[:, ["id"] + address_cols]
addresses.head(2)

,id,region_name,region_type,district_name,district_type,city_name,city_type,settlement_name,settlement_type
0,0,АДЫГЕЯ,РЕСПУБЛИКА,NaN,NaN,МАЙКОП,ГОРОД,ЗАПАДНЫЙ,ПОСЕЛОК
1,1,АДЫГЕЯ,РЕСПУБЛИКА,NaN,NaN,МАЙКОП,ГОРОД,NaN,NaN


In [9]:
addr_count = len(addresses)
addr_count

583282

In [10]:
addresses.isna().sum()

id                      0
region_name             0
region_type             0
district_name      135559
district_type      135559
city_name          418688
city_type          418688
settlement_name    162969
settlement_type    162969
dtype: int64

In [11]:
def join_name_and_type(name, type_):
    if pd.isna(name) or pd.isna(type_):
        return np.nan
    
    prepend_types = (
        "Город", "Республика", "Поселок", "Поселок городского типа", "Рабочий поселок"
    )
    prepend = type_ in prepend_types
    if prepend:
        return f"{type_} {name}"
    
    return f"{name} {type_}"

In [12]:
def preprocess_text_column(c):
    return c.str.upper().str.replace("Ё", "Е")

In [13]:
for option in ("region", "district", "city", "settlement"):
    target_col = f"{option}_type"
    addresses = addresses.merge(
        abbr_to_full,
        how="left",
        left_on=target_col,
        right_on="name",
    )
    addresses[target_col] = addresses["name_full"]
    addresses.drop(columns=abbr_to_full.columns, inplace=True)
    assert len(addresses) == addr_count, (
        f"Number of addresses must not change, but for {target_col} "
        f"the size has changed: {addr_count} -> {len(addresses)}"
    )
    
    parts = [f"{option}_name", f"{option}_type"]
    addresses[option] = addresses[parts].apply(
        lambda row: join_name_and_type(row[parts[0]], row[parts[1]]),
        axis=1
    )
addresses.isna().sum()

id                      0
region_name             0
region_type             0
district_name      135559
district_type      135559
city_name          418688
city_type          418688
settlement_name    162969
settlement_type    164095
region                  0
district           135559
city               418688
settlement         164095
dtype: int64

In [14]:
unique_region_names = addresses["region_name"].dropna().unique()
unique_regions_in_cities = cities["region"].dropna().unique()
region_names_to_cities = pd.DataFrame({
    "region_name": unique_region_names,
    "region_cities": [
        regions.get(region_name).short_name
        for region_name in unique_region_names
    ]
})
region_names_to_cities.head(3)

,region_name,region_cities
0,АДЫГЕЯ,Адыгея
1,БАШКОРТОСТАН,Башкортостан
2,БУРЯТИЯ,Бурятия


In [15]:
unique_regions = addresses["region"].dropna().unique()
unique_regions_in_settlements = settlements["region"].dropna().unique()
regions_to_settlements = pd.DataFrame({
    "region": unique_regions,
    "region_settlements": [
        regions.get(region_name).name
        for region_name in unique_regions
    ]
})
regions_to_settlements.head(3)

,region,region_settlements
0,Республика АДЫГЕЯ,Республика Адыгея
1,Республика БАШКОРТОСТАН,Республика Башкортостан
2,Республика БУРЯТИЯ,Республика Бурятия


In [16]:
addresses = addresses.merge(region_names_to_cities, how="left", on="region_name")
addresses = addresses.merge(regions_to_settlements, how="left", on="region")
addresses.iloc[:, 1:] = addresses.iloc[:, 1:].apply(preprocess_text_column)
assert len(addresses) == addr_count
addresses.head(2)

,id,region_name,region_type,district_name,district_type,city_name,city_type,settlement_name,settlement_type,region,district,city,settlement,region_cities,region_settlements
0,0,АДЫГЕЯ,РЕСПУБЛИКА,NaN,NaN,МАЙКОП,ГОРОД,ЗАПАДНЫЙ,ПОСЕЛОК,РЕСПУБЛИКА АДЫГЕЯ,NaN,ГОРОД МАЙКОП,ПОСЕЛОК ЗАПАДНЫЙ,АДЫГЕЯ,РЕСПУБЛИКА АДЫГЕЯ
1,1,АДЫГЕЯ,РЕСПУБЛИКА,NaN,NaN,МАЙКОП,ГОРОД,NaN,NaN,РЕСПУБЛИКА АДЫГЕЯ,NaN,ГОРОД МАЙКОП,NaN,АДЫГЕЯ,РЕСПУБЛИКА АДЫГЕЯ


In [17]:
std_c = cities[["id", "region", "area", "city", "settlement"]]
cities_from_areas = cities.loc[(cities["area_type"] == "г") & (cities["city"].isna())].copy()
cities_from_areas["city"] = cities_from_areas["area"]
cities_from_areas["area"] = np.nan
cities_from_areas = cities_from_areas[["id", "region", "area", "city", "settlement"]]
std_c = pd.concat((std_c, cities_from_areas))

std_c.iloc[:, 1:] = std_c.iloc[:, 1:].apply(preprocess_text_column)
std_c.head(3)

,id,region,area,city,settlement
0,0,АДЫГЕЯ,NaN,АДЫГЕЙСК,NaN
1,1,АДЫГЕЯ,NaN,МАЙКОП,NaN
2,2,АЛТАЙ,NaN,ГОРНО-АЛТАЙСК,NaN


In [18]:
std_s = settlements.loc[:, ["id", "region", "municipality", "settlement", "type"]]
std_s["type"] = std_s["type"].str.upper()
std_s = std_s.merge(
    abbr_to_full,
    how="left",
    left_on="type",
    right_on="name"
)
std_s["type"] = std_s["name_full"]
std_s.drop(columns=abbr_to_full.columns, inplace=True)
std_s.iloc[:, 1:] = std_s.iloc[:, 1:].apply(preprocess_text_column)
std_s.head(3)

,id,region,municipality,settlement,type
0,0,ОРЛОВСКАЯ ОБЛАСТЬ,БОЛХОВСКИЙ,КОЛОНТАЕВА,ДЕРЕВНЯ
1,1,РЕСПУБЛИКА КРЫМ,АЛУШТА,ПУШКИНО,СЕЛО
2,2,ЛИПЕЦКАЯ ОБЛАСТЬ,ЛЕВ-ТОЛСТОВСКИЙ РАЙОН,БАРЯТИНО,СЕЛО


In [19]:
merge_options = [
    {
        "name": "Settlements by all parts with full district name",
        "addresses": ["region_settlements", "district", "settlement_name", "settlement_type"],
        "standard": ["region", "municipality", "settlement", "type"],
        "type": "settlements"
    },
    {
        "name": "Settlements by all parts with partial district name (no type)",
        "addresses": ["region_settlements", "district_name", "settlement_name", "settlement_type"],
        "standard": ["region", "municipality", "settlement", "type"],
        "type": "settlements",
    },
    {
        "name": "Settlements by all parts with full city name",
        "addresses": ["region_settlements", "city", "settlement_name", "settlement_type"],
        "standard": ["region", "municipality", "settlement", "type"],
        "type": "settlements",
    },
    {
        "name": "Settlements by all parts with partial city name (no type)",
        "addresses": ["region_settlements", "city_name", "settlement_name", "settlement_type"],
        "standard": ["region", "municipality", "settlement", "type"],
        "type": "settlements",
    },
    {
        "name": "Settlements by all parts except for type with full district name",
        "addresses": ["region_settlements", "district", "settlement_name"],
        "standard": ["region", "municipality", "settlement"],
        "type": "settlements",
    },
    {
        "name": "Settlements by all parts except for type with partial district name",
        "addresses": ["region_settlements", "district_name", "settlement_name"],
        "standard": ["region", "municipality", "settlement"],
        "type": "settlements",
    },
    {
        "name": "Settlements by all parts except for type with full city name",
        "addresses": ["region_settlements", "city", "settlement_name"],
        "standard": ["region", "municipality", "settlement"],
        "type": "settlements",
    },
    {
        "name": "Settlements by all parts except for type with partial city name",
        "addresses": ["region_settlements", "city_name", "settlement_name"],
        "standard": ["region", "municipality", "settlement"],
        "type": "settlements",
    },
    {
        "name": "Settlements by region and settlement with type",
        "addresses": ["region_settlements", "settlement_name", "settlement_type"],
        "standard": ["region", "settlement", "type"],
        "type": "settlements",
    },
    {
        "name": "Settlements by region and settlement without type",
        "addresses": ["region_settlements", "settlement_name"],
        "standard": ["region", "settlement"],
        "type": "settlements",
    },
    {
        "name": "Cities by all parts",
        "addresses": ["region_cities", "district_name", "city_name", "settlement_name"],
        "standard": ["region", "area", "city", "settlement"],
        "type": "cities",
    },
    {
        "name": "Cities by all parts except for settlements",
        "addresses": ["region_cities", "district_name", "city_name"],
        "standard": ["region", "area", "city"],
        "type": "cities",
    },
    {
        "name": "Cities by region and city",
        "addresses": ["region_cities", "city_name"],
        "standard": ["region", "city"],
        "type": "cities",
    },
    {
        "name": "Cities by region and district-as-city",
        "addresses": ["region_cities", "city_name"],
        "standard": ["region", "area"],
        "type": "cities",
    },    
]

In [20]:
mappings = []
rest = addresses
orig_cols = addresses.columns
for option in merge_options:
    name = option["name"]
    left_cols = option["addresses"]
    right_cols = option["standard"]
    type_ = option["type"]

    to_merge = rest[orig_cols]
    standard = std_c.copy() if type_ == "cities" else std_s.copy()
    standard.drop_duplicates(subset=right_cols, keep=False, inplace=True)
    if len(right_cols) == 2:
        standard.dropna(subset=right_cols, inplace=True)
    standard.rename(columns={"id": "geo_id"}, inplace=True)
    #print(to_merge.columns)
    #print(standard.columns)
    #print(standard.shape)
    
    size_before = len(to_merge)
    merged = to_merge.merge(
        standard,
        how="left",
        left_on=left_cols,
        right_on=right_cols,
        suffixes=("", "_x")
    )
    
    size_after = len(merged)
    assert size_before == size_after
    
    mapped = merged.loc[merged["geo_id"].notna(), ["id", "geo_id"]]
    mapped["type"] = type_[0]
    if len(mapped) > 0:
        mappings.append(mapped)
        
    rest = merged.loc[merged["geo_id"].isna()]
    
    print(f"Option {name}: found {len(mapped)} matches, {len(rest)} records left")

Option Settlements by all parts with full district name: found 220246 matches, 363036 records left
Option Settlements by all parts with partial district name (no type): found 81626 matches, 281410 records left
Option Settlements by all parts with full city name: found 686 matches, 280724 records left
Option Settlements by all parts with partial city name (no type): found 2443 matches, 278281 records left
Option Settlements by all parts except for type with full district name: found 40214 matches, 238067 records left
Option Settlements by all parts except for type with partial district name: found 8640 matches, 229427 records left
Option Settlements by all parts except for type with full city name: found 443 matches, 228984 records left
Option Settlements by all parts except for type with partial city name: found 481 matches, 228503 records left
Option Settlements by region and settlement with type: found 31685 matches, 196818 records left
Option Settlements by region and settlement wit

In [21]:
addr_to_geo = pd.concat(mappings)
addr_to_geo.head(3)

,id,geo_id,type
13,13,45854.0,s
15,15,125001.0,s
16,16,124914.0,s


In [22]:
rsmp_with_geo_ids = rsmp.merge(addr_to_geo, how="left")
rsmp_with_geo_ids[["geo_id"]].isna().sum()

geo_id    26112
dtype: int64

In [23]:
geodata_s = settlements[["id", "region", "municipality", "settlement", "type", "oktmo", "longitude_dd", "latitude_dd"]].copy()
geodata_s["geosource_type"] = "s"
geodata_s.rename(columns={
    "id": "geo_id",
    "municipality": "area",
    "type": "settlement_type",
    "longitude_dd": "lon",
    "latitude_dd": "lat",
    "geosource_type": "type",
}, inplace=True)
geodata_s["region"] = geodata_s["region"].apply(lambda x: regions.get(x).name)
geodata_s.head()

,geo_id,region,area,settlement,settlement_type,oktmo,lon,lat,type
0,0,Орловская область,Болховский,Колонтаева,д,5.460442e+10,35.910000,53.368611,s
1,1,Республика Крым,Алушта,Пушкино,с,3.570300e+10,34.340833,44.595833,s
2,2,Липецкая область,Лев-Толстовский район,Барятино,с,4.263641e+10,39.503889,53.262778,s
3,3,Тверская область,Селижаровский район,Хилово,д,2.865043e+10,33.419167,56.905556,s
4,4,Томская область,Парабельский район,Басмасово,д,6.964444e+10,82.044444,58.636667,s


In [24]:
def join_area_and_type(a, t):
    if pd.isna(a) or pd.isna(t):
        return np.nan
    
    if t == "г":
        return f"Город {a}"
    elif t == "р-н":
        return f"{a} район"
    elif t == "у":
        return f"{a} улус"
    else:
        return a
    
    
geodata_c = cities[["id", "region", "region_type", "area", "area_type", "city", "city_type", "settlement", "settlement_type", "oktmo", "geo_lat", "geo_lon"]].copy()
geodata_c["settlement"] = cities["settlement"].combine_first(cities["city"]).combine_first(cities["area"]).reset_index(drop=True)
geodata_c.loc[geodata_c["area_type"] == "г", "area"] = np.nan
geodata_c["area"] = geodata_c[["area", "area_type"]].apply(lambda x: join_area_and_type(x[0], x[1]), axis=1)

regions_c_s = pd.DataFrame({
    "region": geodata_c["region"].unique(),
    "region_norm": [
        regions.get(r).name
        for r in geodata_c["region"].unique()]
})

geodata_c = geodata_c.merge(regions_c_s, how="left")
geodata_c["region"] = geodata_c["region_norm"]
geodata_c["geosource_type"] = "c"
geodata_c["settlement_type"] = "г"
geodata_c.rename(columns={
    "id": "geo_id",
    "geo_lat": "lat",
    "geo_lon": "lon",
    "geosource_type": "type",
}, inplace=True)
geodata_c.drop(columns=["region_type", "area_type", "city", "city_type", "region_norm"], inplace=True)
geodata_c.head(3)

,geo_id,region,area,settlement,settlement_type,oktmo,lat,lon,type
0,0,Республика Адыгея,NaN,Адыгейск,г,7.970300e+10,44.878414,39.190289,c
1,1,Республика Адыгея,NaN,Майкоп,г,7.970100e+10,44.609827,40.100661,c
2,2,Республика Алтай,NaN,Горно-Алтайск,г,8.470100e+10,51.958103,85.960324,c


In [25]:
geodata = pd.concat((geodata_c, geodata_s))
geodata.shape

(157050, 9)

In [26]:
rsmp = rsmp_with_geo_ids.merge(geodata, how="left", on=["geo_id", "type"])
assert len(rsmp) == addr_count
rsmp.head(3)

,kind,category,tin,reg_number,first_name,last_name,patronymic,org_name,org_short_name,region_code,region_name,region_type,district_name,district_type,city_name,city_type,settlement_name,settlement_type_x,activity_code_main,start_date,end_date,id,geo_id,type,region,area,settlement,settlement_type_y,oktmo,lat,lon
0,1,1,0105035949,1020100709111,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "" ЛЮЦ...",ЛЮЦЕРНА АГРИКОЛ ООО,1,АДЫГЕЯ,РЕСПУБЛИКА,NaN,NaN,МАЙКОП,ГОРОД,ЗАПАДНЫЙ,ПОСЕЛОК,01.47,2016-08-10,2023-11-10,0,94088.0,s,Республика Адыгея,Городской округ Майкоп,Западный,п,7.970100e+10,44.631667,40.071944
1,1,1,0105075540,NaN,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ДАГМАЙ""","ООО ""ДАГМАЙ""",1,АДЫГЕЯ,РЕСПУБЛИКА,NaN,NaN,МАЙКОП,ГОРОД,NaN,NaN,03.2,2017-08-10,2020-07-10,1,1.0,c,Республика Адыгея,NaN,Майкоп,г,7.970100e+10,44.609827,40.100661
2,1,1,0218000256,1020201626686,NaN,NaN,NaN,СЕЛЬСКОХОЗЯЙСТВЕННЫЙ ПРОИЗВОДСТВЕННЫЙ КООПЕРАТ...,СПК ИМЕНИ САЛАВАТА,2,БАШКОРТОСТАН,РЕСПУБЛИКА,БУРЗЯНСКИЙ,РАЙОН,NaN,NaN,КУТАНОВО,ДЕРЕВНЯ,01.41,2016-08-10,2023-11-10,2,6729.0,s,Республика Башкортостан,Бурзянский,Кутаново,д,8.061942e+10,52.983333,57.032778


In [27]:
rsmp.isna().sum()

kind                       0
category                   0
tin                        0
reg_number            191826
first_name            126094
last_name             126093
patronymic            128890
org_name              457189
org_short_name        458651
region_code                0
region_name                0
region_type                0
district_name         135559
district_type         135559
city_name             418688
city_type             418688
settlement_name       162969
settlement_type_x     162969
activity_code_main         0
start_date                 0
end_date                   0
id                         0
geo_id                 26112
type                   26112
region                 26112
area                  151968
settlement             39415
settlement_type_y      26112
oktmo                  26270
lat                    26112
lon                    26112
dtype: int64

In [28]:
addresses["address_raw"] = addresses.loc[:, "region_name":"settlement_type"].apply(lambda x: " / ".join(x.fillna("").to_list()), axis=1)

In [29]:
regions_to_settlements["region"] = regions_to_settlements["region"].str.upper()
regions_to_settlements.rename(columns={"region_settlements": "region_norm"}, inplace=True)
addresses = addresses.merge(
    regions_to_settlements,
    how="left",
)
assert len(addresses) == addr_count
addresses.head(2)

,id,region_name,region_type,district_name,district_type,city_name,city_type,settlement_name,settlement_type,region,district,city,settlement,region_cities,region_settlements,address_raw,region_norm
0,0,АДЫГЕЯ,РЕСПУБЛИКА,NaN,NaN,МАЙКОП,ГОРОД,ЗАПАДНЫЙ,ПОСЕЛОК,РЕСПУБЛИКА АДЫГЕЯ,NaN,ГОРОД МАЙКОП,ПОСЕЛОК ЗАПАДНЫЙ,АДЫГЕЯ,РЕСПУБЛИКА АДЫГЕЯ,АДЫГЕЯ / РЕСПУБЛИКА / / / МАЙКОП / ГОРОД / З...,Республика Адыгея
1,1,АДЫГЕЯ,РЕСПУБЛИКА,NaN,NaN,МАЙКОП,ГОРОД,NaN,NaN,РЕСПУБЛИКА АДЫГЕЯ,NaN,ГОРОД МАЙКОП,NaN,АДЫГЕЯ,РЕСПУБЛИКА АДЫГЕЯ,АДЫГЕЯ / РЕСПУБЛИКА / / / МАЙКОП / ГОРОД / /,Республика Адыгея


In [30]:
addresses.isna().sum()

id                         0
region_name                0
region_type                0
district_name         135559
district_type         135559
city_name             418688
city_type             418688
settlement_name       162969
settlement_type       164095
region                     0
district              135559
city                  418688
settlement            164095
region_cities             46
region_settlements        47
address_raw                0
region_norm               47
dtype: int64

In [31]:
rsmp = rsmp.merge(
    addresses[["id", "address_raw", "region_norm"]],
    how="left",
    on="id",
)
assert len(rsmp) == addr_count
rsmp.drop(
    columns=[
        "region_name", "region_type", "district_name", "district_type",
        "city_name", "city_type", "settlement_name", "geo_id",
        "type", "settlement_type_x"],
    inplace=True
)
rsmp.head(2)

,kind,category,tin,reg_number,first_name,last_name,patronymic,org_name,org_short_name,region_code,activity_code_main,start_date,end_date,id,region,area,settlement,settlement_type_y,oktmo,lat,lon,address_raw,region_norm
0,1,1,0105035949,1020100709111,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "" ЛЮЦ...",ЛЮЦЕРНА АГРИКОЛ ООО,1,01.47,2016-08-10,2023-11-10,0,Республика Адыгея,Городской округ Майкоп,Западный,п,7.970100e+10,44.631667,40.071944,АДЫГЕЯ / РЕСПУБЛИКА / / / МАЙКОП / ГОРОД / З...,Республика Адыгея
1,1,1,0105075540,NaN,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ДАГМАЙ""","ООО ""ДАГМАЙ""",1,03.2,2017-08-10,2020-07-10,1,Республика Адыгея,NaN,Майкоп,г,7.970100e+10,44.609827,40.100661,АДЫГЕЯ / РЕСПУБЛИКА / / / МАЙКОП / ГОРОД / /,Республика Адыгея


In [32]:
rsmp["region"] = rsmp["region"].combine_first(rsmp["region_norm"])

In [33]:
rsmp.rename(columns={"settlement_type_y": "settlement_type"}, inplace=True)

In [34]:
cols_to_check_for_duplicates = [
    "kind",
    "category",
    "tin",
    "reg_number",
    "first_name",
    "last_name",
    "patronymic",
    "org_name",
    "org_short_name",
    "activity_code_main",
    "region",
    "area",
    "settlement",
    "settlement_type",
    "oktmo",
    "lat",
    "lon",
]
duplicates_indices = rsmp.duplicated(
    subset=cols_to_check_for_duplicates,
    keep=False
)
duplicates_cleaned = (
    rsmp.loc[duplicates_indices]
    .sort_values("start_date")
    .groupby(cols_to_check_for_duplicates, dropna=False)
    .agg({"id": "first", "address_raw": "first", "start_date": "first", "end_date": "last"})
    .reset_index()
)
duplicates_cleaned.head(3)

,kind,category,tin,reg_number,first_name,last_name,patronymic,org_name,org_short_name,activity_code_main,region,area,settlement,settlement_type,oktmo,lat,lon,id,address_raw,start_date,end_date
0,1,1,0205004493,1020201255216,NaN,NaN,NaN,"ТОВАРИЩЕСТВО НА ВЕРЕ ""УСМАН""","ТВ ""УСМАН""",01.11.1,Республика Башкортостан,NaN,NaN,NaN,NaN,NaN,NaN,548307,БАШКОРТОСТАН / РЕСПУБЛИКА / АУРГАЗИНСКИЙ / РАЙ...,2019-08-10,2023-11-10
1,1,1,0245007290,1030202315373,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ШТИЛЬ""","ООО ""ШТИЛЬ""",03.12,Республика Башкортостан,NaN,NaN,NaN,NaN,NaN,NaN,411318,БАШКОРТОСТАН / РЕСПУБЛИКА / УФИМСКИЙ / РАЙОН /...,2016-08-10,2023-11-10
2,1,1,0259010790,1100259000225,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ТОЛПАР""","ООО ""ТОЛПАР""",01.41,Республика Башкортостан,Давлекановский,Кадыргулово,д,8.062242e+10,54.152778,55.564722,5977,БАШКОРТОСТАН / РЕСПУБЛИКА / ДАВЛЕКАНОВСКИЙ / Р...,2016-08-10,2023-11-10


In [35]:
rsmp = pd.concat((rsmp.loc[~duplicates_indices], duplicates_cleaned))
rsmp.shape

(572963, 23)

In [36]:
product_cols = [
    "id",
    "tin",
    "reg_number",
    "kind",
    "category",
    "first_name",
    "last_name",
    "patronymic",
    "org_name",
    "org_short_name",
    "activity_code_main",
    "region",
    "area",
    "settlement",
    "settlement_type",
    "oktmo",
    "lat",
    "lon",
    "address_raw",
    "start_date",
    "end_date",
]
rsmp_product = rsmp[product_cols]
rsmp_product.isna().sum()

id                         0
tin                        0
reg_number            189135
kind                       0
category                   0
first_name            122005
last_name             122004
patronymic            124750
org_name              450959
org_short_name        452343
activity_code_main         0
region                    47
area                  146251
settlement             37112
settlement_type        23845
oktmo                  24003
lat                    23845
lon                    23845
address_raw                0
start_date                 0
end_date                   0
dtype: int64

In [37]:
assert len(rsmp_product) == len(rsmp_product.drop_duplicates(cols_to_check_for_duplicates))

In [38]:
rsmp_product.shape

(572963, 21)

In [38]:
rsmp_product.to_csv("rsmp/reestr_group_A/data_product.csv", index=False)

In [41]:
rsmp_product.loc[rsmp_product["lat"].isna()]

,id,tin,reg_number,kind,category,first_name,last_name,patronymic,org_name,org_short_name,activity_code_main,region,area,settlement,settlement_type,oktmo,lat,lon,address_raw,start_date,end_date
84,84,1714007042,1201700001469,1,1,NaN,NaN,NaN,СЕЛЬСКОХОЗЯЙСТВЕННЫЙ ПРОИЗВОДСТВЕННЫЙ КООПЕРАТ...,"СПК ""УЛУГ-ХАЯ""",01.45,Республика Тыва,NaN,NaN,NaN,NaN,NaN,NaN,ТЫВА / РЕСПУБЛИКА / УЛУГ-ХЕМСКИЙ / РАЙОН / / ...,2020-11-10,2023-11-10
103,103,2103005131,NaN,1,1,NaN,NaN,NaN,СЕЛЬСКОХОЗЯЙСТВЕННЫЙ СБЫТОВОЙ СНАБЖЕНЧЕСКИЙ ПЕ...,"СССППК ""АГРО-СОЮЗ""",01.42,Чувашская республика - Чувашия,NaN,NaN,NaN,NaN,NaN,NaN,ЧУВАШСКАЯ РЕСПУБЛИКА - / ЧУВАШИЯ / БАТЫРЕВСКИЙ...,2019-07-10,2020-03-10
104,104,2106007426,NaN,1,1,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ВОЛЖ...","ООО ""ВОЛЖСКИЙ КАРТОФЕЛЬ""",01.1,Чувашская республика - Чувашия,NaN,NaN,NaN,NaN,NaN,NaN,ЧУВАШСКАЯ РЕСПУБЛИКА - / ЧУВАШИЯ / КАНАШСКИЙ /...,2016-08-10,2018-07-10
105,105,2113004176,NaN,1,1,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ШАНС""","ООО ""ШАНС""",01.49.1,Чувашская республика - Чувашия,NaN,NaN,NaN,NaN,NaN,NaN,ЧУВАШСКАЯ РЕСПУБЛИКА - / ЧУВАШИЯ / ПОРЕЦКИЙ / ...,2016-08-10,2018-02-10
106,106,2115904013,NaN,1,1,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АГРО...","ООО ""АГРОФИРМА ""КИБЕКСИ""",01.11.1,Чувашская республика - Чувашия,NaN,NaN,NaN,NaN,NaN,NaN,ЧУВАШСКАЯ РЕСПУБЛИКА - / ЧУВАШИЯ / ЦИВИЛЬСКИЙ ...,2019-08-10,2020-11-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9337,548249,693203334209,312690613700010,3,1,АНДРЕЙ,ТРОПИН,ВАЛЕРЬЕВИЧ,NaN,NaN,01.1,Тверская область,NaN,NaN,NaN,NaN,NaN,NaN,ТВЕРСКАЯ / ОБЛАСТЬ / РАМЕШКОВСКИЙ / РАЙОН / НЕ...,2021-08-10,2023-11-10
9353,227030,761200132081,304761218800059,3,1,ЕЛЕНА,ВАСИЛЕВСКАЯ,ВЛАДИМИРОВНА,NaN,NaN,01.4,Ярославская область,NaN,NaN,NaN,NaN,NaN,NaN,ЯРОСЛАВСКАЯ / ОБЛАСТЬ / УГЛИЧСКИЙ / РАЙОН / /...,2021-08-10,2023-11-10
9359,446089,780504089015,320784700128631,3,1,ФАРИД,МАХМУДОВ,МЕДЖИД ОГЛЫ,NaN,NaN,01.45,Ленинградская область,NaN,NaN,NaN,NaN,NaN,NaN,ЛЕНИНГРАДСКАЯ / ОБЛАСТЬ / ЛОМОНОСОВСКИЙ / РАЙО...,2021-10-10,2023-04-10
9361,332888,800100935345,320753600019809,3,1,ЭРДЭМ,НИМАЕВ,ПАЛАМОВИЧ,NaN,NaN,01.45,Забайкальский край,NaN,NaN,NaN,NaN,NaN,NaN,ЗАБАЙКАЛЬСКИЙ / КРАЙ / АГИНСКИЙ / РАЙОН / / ...,2021-08-10,2023-11-10


In [45]:
rsmp_product.iloc[:1000, ].to_csv("rsmp/csv/data_product_demo.csv", index=False)